In [1]:
import requests
import datetime
import pandas as pd
import solcast
import configparser
import pytz
from pvlive_api import PVLive
import io

In [28]:
config = configparser.ConfigParser()
if not config.has_section("APIKEY_SOLCAST"):
    config.add_section("APIKEY_SOLCAST")
    config.set("APIKEY_SOLCAST", "key", "bmDCPHfGBni1xCX8uBxuGJmRhRLbLaVC")

if not config.has_section("APIKEY_BMRS"):
    config.add_section("APIKEY_BMRS")
    config.set("APIKEY_BMRS", "key", "ngfxa9q76mps023")

if not config.has_section("APIKEY_OPENWEATHER"):
    config.add_section("APIKEY_OPENWEATHER")
    config.set("APIKEY_OPENWEATHER", "key", "65aa4455df560428020b8579a28ad042")

if not config.has_section("APIKEY_WEBNEWS"):
    config.add_section("APIKEY_WEBNEWS")
    config.set("APIKEY_WEBNEWS", "key", "89e48674-0978-4c78-a753-cf89ddbf0749")

    

with open("API_keys.ini", 'w') as configfile:
    config.write(configfile)

In [3]:
config = configparser.ConfigParser()		
config.read("API_keys.ini")
apikey_solcast = config['APIKEY_SOLCAST']
apikey_bmrs = config['APIKEY_BMRS']
apikey_openweather = config['APIKEY_OPENWEATHER']

### `https://api0.solar.sheffield.ac.uk/pvlive/v2/`,
### `https://api.bmreports.com/BMRS/FUELINSTHHCUR/V1?APIKey=16hudca3onmwxcy&ServiceType=xml`,
### `https://api.bmreports.com/BMRS/FUELHH/V1?APIKey=16hudca3onmwxcy&FromDate=${todayDate}&ToDate=${todayDate}&ServiceType=xml`,
### `https://api0.solar.sheffield.ac.uk/pvlive/v2?start=${yestDate}T23:30:00&end=${todayDate}T23:59:59`,
### `https://api.bmreports.com/BMRS/INDOITSDO/V1?APIKey=16hudca3onmwxcy&FromDate=${todayDate}&ToDate=${todayDate}&ServiceType=xml'

# Solcast

In [ ]:


url = "https://api.solcast.com.au/world_radiation/estimated_actuals?format=json"


querystring = {"api_key":"bmDCPHfGBni1xCX8uBxuGJmRhRLbLaVC","longitude":"-0.127758","latitude":"51.507351", "hours":"168"}

headers={'Content-Type': 'application/json'}

response = requests.request("GET", url, headers=headers, params=querystring)
print(response.json())


In [34]:

url = 'https://api.solcast.com.au/world_radiation/estimated_actuals?format=json'
querystring = {"api_key" : apikey_solcast['key'],"longitude":"-0.127758","latitude":"51.507351", "hours":"168"}
res = requests.get(url, params=querystring, headers={'Content-Type': 'application/json'})
data = res.json()
print(data)
forecast = data["estimated_actuals"][0]["ghi"]

print('estimated_actuals: {} degree'.format(forecast))

{'response_status': {'error_code': 'TooManyRequests', 'message': 'You have exceeded your free daily limit.', 'errors': []}}


KeyError: 'estimated_actuals'

In [32]:

url = 'https://api.solcast.com.au/world_pv_power/estimated_actuals?format=json'
querystring = {"api_key" : apikey_solcast['key'],"latitude":"51.507351","longitude":"-0.127758","capacity":"5","tilt":"42","azimuth":"180","hours":"168"}
res = requests.get(url, params=querystring, headers={'Content-Type': 'application/json'})
data = res.json()
print(data)
forecast = data["estimated_actuals"][0]["ghi"]

print('estimated_actuals: {} degree'.format(forecast))

{'response_status': {'error_code': 'TooManyRequests', 'message': 'You have exceeded your free daily limit.', 'errors': []}}


KeyError: 'estimated_actuals'

In [4]:
response = solcast.get_radiation_forecasts(-35, 149, apikey_solcast['key'])

In [5]:
response.forecasts[0]

{'ghi': 127,
 'ghi90': 128,
 'ghi10': 117,
 'ebh': 78,
 'dni': 435,
 'dni10': 374,
 'dni90': 437,
 'dhi': 49,
 'air_temp': 6,
 'zenith': 80,
 'azimuth': 56,
 'cloud_opacity': 0,
 'period_end': datetime.datetime(2021, 7, 21, 6, 30, tzinfo=<isodate.tzinfo.Utc object at 0x000001B9104EBC10>),
 'period': datetime.timedelta(seconds=1800)}

# PV_live

In [29]:
pvl = PVLive()

In [35]:
pvl.latest(entity_id=23)	

(23, '2021-07-21T14:30:00Z', 120.01024)

In [42]:
datetime.datetime(2020, 1, 1, 0, 30,0,0)

datetime.datetime(2020, 1, 1, 0, 30)

In [44]:
pvl.between(start=datetime.datetime(2020, 1, 1, 0, 30,0,0, tzinfo=pytz.utc), end=datetime.datetime(2021, 1, 1, 0, 30,0,0, tzinfo=pytz.utc), dataframe=True)

,pes_id,datetime_gmt,generation_mw
0,0,2020-01-31 00:30:00+00:00,0.00
1,0,2020-01-31 00:00:00+00:00,0.00
2,0,2020-01-30 23:30:00+00:00,0.00
3,0,2020-01-30 23:00:00+00:00,0.00
4,0,2020-01-30 22:30:00+00:00,0.00
...,...,...,...
17564,0,2020-12-26 08:30:00+00:00,2.28
17565,0,2020-12-26 08:00:00+00:00,0.00
17566,0,2020-12-26 07:30:00+00:00,0.00
17567,0,2020-12-26 07:00:00+00:00,0.00


# BMRS

In [8]:

url = 'https://api.bmreports.com/BMRS/B1720/v1?ServiceType=csv'
querystring = {"APIKey" : apikey_bmrs['key'], "SettlementDate":"2019-12-31", "Period":"1"}
res = requests.get(url, params=querystring, headers={'Content-Type': 'application/csv'})
print(res.text)

*
*
*Amount Of Balancing Reserves Under Contract (B1720) Data
*
*Time Series ID,Business Type,Control Area,Market Agreement Type,Power System Resource  Type,Flow Direction,Settlement Date,Settlement Period,Quantity (MAW),Doc Status,Curve Type,Resolution,Document Type,Process Type,Active Flag,Document ID,Document RevNum
NGET-EMFIP-ABUC-TS-00000006,Frequency containment reserve,10YGB----------A,Daily,Generation,DOWN,2019-12-31,1,329,,Variable sized Block,PT30M,Contracted reserves,Contracted,Y,NGET-EMFIP-ABUC-00000705,1
NGET-EMFIP-ABUC-TS-00000004,Replacement reserve,10YGB----------A,Daily,Generation,UP,2019-12-31,1,2173,,Variable sized Block,PT30M,Contracted reserves,Contracted,Y,NGET-EMFIP-ABUC-00000701,1
NGET-EMFIP-ABUC-TS-00000006,Frequency containment reserve,10YGB----------A,Daily,Generation,UP,2019-12-31,1,368,,Variable sized Block,PT30M,Contracted reserves,Contracted,Y,NGET-EMFIP-ABUC-00000703,1
<EOF>


In [22]:
url = 'https://api.bmreports.com/BMRS/TEMP/v1?ServiceType=csv'
querystring = {"APIKey" : apikey_bmrs['key'],"FromDate":"2019-12-31","ToDate":"2020-01-01"}
res = requests.get(url, params=querystring, headers={'Content-Type': 'application/csv'})
print(res.content.decode('utf-8'))

HDR,TEMPERATURE DATA
TEMP,20191231,6.5,6.0,1.4,8.9
TEMP,20200101,6.2,5.8,1.4,8.7
FTR,2


In [23]:
r = res.content
rawData = pd.read_csv(io.StringIO(r.decode('utf-8')))
rawData.head()

HDR  TEMPERATURE DATA
TEMP 20191231 6.5 6.0  1.4               8.9
     20200101 6.2 5.8  1.4               8.7
FTR  2        NaN NaN  NaN               NaN

In [27]:
rawData.shape

(3, 2)

In [25]:
rawData.rename(columns=['Type', 'Datetime', 'tep1', 'temp2', 'temp3', 'temp4'])

TypeError: 'list' object is not callable

In [11]:
url = 'https://api.bmreports.com/BMRS/B0810/v1?ServiceType=csv'
querystring = {"APIKey" : apikey_bmrs['key'],"Year":"2021"}
res = requests.get(url, params=querystring, headers={'Content-Type': 'application/csv'})
print(res.text)

*
*
*Year Ahead Forecast Margin Service (B0810) Data
*
*Business Type,Time Series ID,Year,Quantity ,Secondary Quantity (MAW),Document Type,Process Type,Curve Type,Resolution,Unit of Measure,Active Flag,Document ID,Document RevNum
positive forecast margin,NGET-EMFIP-YAFM-TS-00000001,2021,4410,,Load forecast margin,Year ahead,Sequential fixed size block,P1Y,Mega watt,Y,NGET-EMFIP-YAFM-00687691,1
<EOF>


In [12]:
url = 'https://api.bmreports.com/BMRS/B1410/v1?ServiceType=csv'
querystring = {"APIKey" : apikey_bmrs['key'],"Year":"2021"}
res = requests.get(url, params=querystring, headers={'Content-Type': 'application/csv'})
print(res.text)

*
*
*postInstalledGenerationCapacityAggregated (B1410) Data
*
*Document Type,Business Type,Process Type,Time Series ID,Quantity,Resolution,Year,Power System Resource  Type,Active Flag,Document ID,Document RevNum
Installed generation per type,Installed generation,Year ahead,NGET-EMFIP-IGCA-TS-00000012,12937,P1Y,2021,"Wind Onshore",Y,NGET-EMFIP-IGCA-00686934,1
Installed generation per type,Installed generation,Year ahead,NGET-EMFIP-IGCA-TS-00000011,12160,P1Y,2021,"Wind Offshore",Y,NGET-EMFIP-IGCA-00686934,1
Installed generation per type,Installed generation,Year ahead,NGET-EMFIP-IGCA-TS-00000010,13378,P1Y,2021,"Solar",Y,NGET-EMFIP-IGCA-00686934,1
Installed generation per type,Installed generation,Year ahead,NGET-EMFIP-IGCA-TS-00000009,1900,P1Y,2021,"Other renewable",Y,NGET-EMFIP-IGCA-00686934,1
Installed generation per type,Installed generation,Year ahead,NGET-EMFIP-IGCA-TS-00000008,2808,P1Y,2021,"Other",Y,NGET-EMFIP-IGCA-00686934,1
Installed generation per type,Installed generation,Year

In [13]:
url = 'https://api.bmreports.com/BMRS/B1630/v1?ServiceType=csv'
querystring = {"APIKey" : apikey_bmrs['key'],"SettlementDate":"2020-12-30", "Period":"5"}
res = requests.get(url, params=querystring, headers={'Content-Type': 'application/csv'})
print(res.text)

*
*
*Actual Or Estimated Wind And Solar Power Generation(B1630) Data
*
*Time Series ID,Business Type,Power System Resource  Type,Settlement Date,Settlement Period,Quantity ,Document Type,Process Type,Curve Type,Resolution,Active Flag,Document ID,Document RevNum
NGET-EMFIP-AGWS-TS-20950021,Solar generation,"Solar",2020-12-30,5,0,Wind and solar generation,Realised,Sequential fixed size block,PT30M,Y,NGET-EMFIP-AGWS-20765739,1
NGET-EMFIP-AGWS-TS-20950022,Wind generation,"Wind Offshore",2020-12-30,5,3181.701,Wind and solar generation,Realised,Sequential fixed size block,PT30M,Y,NGET-EMFIP-AGWS-20765739,1
NGET-EMFIP-AGWS-TS-20950023,Wind generation,"Wind Onshore",2020-12-30,5,1838.74,Wind and solar generation,Realised,Sequential fixed size block,PT30M,Y,NGET-EMFIP-AGWS-20765739,1
<EOF>


## Energy generated by Type

In [14]:
url = 'https://api.bmreports.com/BMRS/B1620/v1?ServiceType=csv'
querystring = {"APIKey" : apikey_bmrs['key'],"SettlementDate":"2020-12-31", "Period":"1", "NGCBMUnitID": "DRAXX-2"}
res = requests.get(url, params=querystring, headers={'Content-Type': 'application/csv'})
print(res.text)

*
*
*Actual Aggregated Generation Per Type (B1620) Data
*
*Document Type,Business Type,Process Type,Time Series ID,Quantity,Curve Type,Resolution,Settlement Date,Settlement Period,Power System Resource  Type,Active Flag,Document ID,Document RevNum
Actual generation per type,Solar generation,Realised,NGET-EMFIP-AGPT-TS-21711171,0,Sequential fixed size block,PT30M,2020-12-31,1,"Solar",Y,NGET-EMFIP-AGPT-06381276,1
Actual generation per type,Wind generation,Realised,NGET-EMFIP-AGPT-TS-21711172,1283.384,Sequential fixed size block,PT30M,2020-12-31,1,"Wind Offshore",Y,NGET-EMFIP-AGPT-06381276,1
Actual generation per type,Wind generation,Realised,NGET-EMFIP-AGPT-TS-21711173,1563.476,Sequential fixed size block,PT30M,2020-12-31,1,"Wind Onshore",Y,NGET-EMFIP-AGPT-06381276,1
Actual generation per type,Production,Realised,NGET-EMFIP-AGPT-TS-21711174,161,Sequential fixed size block,PT30M,2020-12-31,1,"Other",Y,NGET-EMFIP-AGPT-06381276,1
Actual generation per type,Production,Realised,NGET-EMFIP-AGP

In [15]:
url = 'https://api.bmreports.com/BMRS/NOU2T14D/v1?ServiceType=csv'
querystring = {"APIKey" : apikey_bmrs['key']}
res = requests.get(url, params=querystring, headers={'Content-Type': 'application/csv'})
print(res.text)

HDR,NATIONAL OUTPUT USABLE MW BASED ON OC2 (2-14 DAYS) DATA
NOU2T14D,20210721060000,N,20210723,40432
NOU2T14D,20210721060000,N,20210724,37097
NOU2T14D,20210721060000,N,20210725,37642
NOU2T14D,20210721060000,N,20210726,38014
NOU2T14D,20210721060000,N,20210727,38703
NOU2T14D,20210721060000,N,20210728,38670
NOU2T14D,20210721060000,N,20210729,40291
NOU2T14D,20210721060000,N,20210730,40123
NOU2T14D,20210721060000,N,20210731,37709
NOU2T14D,20210721060000,N,20210801,38433
NOU2T14D,20210721060000,N,20210802,37463
NOU2T14D,20210721060000,N,20210803,37583
NOU2T14D,20210721060000,N,20210804,37501
FTR,13


In [18]:
url = 'https://api.bmreports.com/BMRS/FOU2T52W/v1?ServiceType=csv'
querystring = {"APIKey" : apikey_bmrs['key']}
res = requests.get(url, params=querystring, headers={'Content-Type': 'application/csv'})
print(res.text)

HDR,NATIONAL OUTPUT USABLE MW BASED ON OC2 (2-52 WEEKS) DATA BY FUEL TYPE
FOU2T52W,BIOMASS,20210721070900,N,31,2021,3276
FOU2T52W,CCGT,20210721070900,N,31,2021,19376
FOU2T52W,COAL,20210721070900,N,31,2021,3282
FOU2T52W,INTELEC,20210721070900,N,31,2021,0
FOU2T52W,INTEW,20210721070900,N,31,2021,500
FOU2T52W,INTFR,20210721070900,N,31,2021,2000
FOU2T52W,INTIFA2,20210721070900,N,31,2021,1100
FOU2T52W,INTIRL,20210721070900,N,31,2021,500
FOU2T52W,INTNED,20210721070900,N,31,2021,1000
FOU2T52W,INTNEM,20210721070900,N,31,2021,1046
FOU2T52W,INTNSL,20210721070900,N,31,2021,0
FOU2T52W,NPSHYD,20210721070900,N,31,2021,922
FOU2T52W,NUCLEAR,20210721070900,N,31,2021,5256
FOU2T52W,OCGT,20210721070900,N,31,2021,1066
FOU2T52W,OTHER,20210721070900,N,31,2021,100
FOU2T52W,PS,20210721070900,N,31,2021,2016
FOU2T52W,WIND,20210721070900,N,31,2021,5339
FOU2T52W,BIOMASS,20210721070900,N,32,2021,3276
FOU2T52W,CCGT,20210721070900,N,32,2021,21365
FOU2T52W,COAL,20210721070900,N,32,2021,3774
FOU2T52W,INTELEC,20210721070